# TP3 - Ejercicio 3
### Enunciado

##### Segmentación de Imagenes en color:
Considere la imagen cow.jpg y las imágenes muestra: vaca.jpg, cielo.jpg y pasto.jpg correspondientes a las clases dentro de la imagen “vaca”, “cielo” y “pasto”, respectivamente.

a)  Construir un conjunto de datos para entrenamiento, indicando para cada muestra a qué clase pertenece.

b)  Dividir aleatoriamente el conjunto de datos en dos conjuntos, uno de entrenamiento y uno de prueba.

c)  Utilizar el método SVM para clasificar los pixels del conjunto de prueba, entrenando con el conjunto de entrenamiento. Utilizar diferentes núcleos y diferentes valores del parámetro C. Construir la matriz de confusión para cada caso.

d)  ¿Cuál es el núcleo que da mejores resultados? Pensar una justificación teórica para la respuesta.

e)  Con el mismo método ya entrenado clasificar todos los pixels de la imagen.

f)  Con el mismo método ya entrenado clasificar todos los pixels de otra imagen.

In [25]:
import os
import matplotlib.image as img
import matplotlib.pyplot as plt

print("Module and package import success")

Module and package import success


In [26]:
dir_path = os.path.dirname(os.path.realpath("e3.ipynb"))
DEFAULT_FILEPATH = f"{dir_path}/../tp3/data/"

print("Variable setting and data import success")

Variable setting and data import success


In [27]:
pictures = []
picture_names = ["cielo", "cow", "pasto", "vaca"]

for pic in picture_names:
    pictures.append(img.imread(DEFAULT_FILEPATH + pic + ".jpg"))

for pic in pictures:
    plt.imshow(pic)